In [1]:
import pandas as pd
import plotly.plotly as py

In [ ]:
%load_ext sql
%sql sqlite://
%config SqlMagic.feedback = False #to turn off sqlite reports

In [ ]:
## Load csv files into SQL via pandas #
 # All raw data save to table ALLPOST ##
import pandas as pd
location_list=["Toronto_ON", "Montreal_QC", "Vancouver_BC", "Waterloo_ON", "New-York_NY", "Boston_MA", "Washington_DC", 
               "Seattle_WA", "San-Francisco_CA", "Austin_TX", "San-Jose_CA", "Raleigh_NC", "Houston_TX", "Portland_OR", 
               "Atlanta_GA", "San-Diego_CA", "Denver_CO", "Minneapolis_MN", "Saint-Louis_MO"]
%sql DROP TABLE IF EXISTS ALLPOST
%sql CREATE TABLE ALLPOST (keyword varchar, location varchar, link varchar, post_text varchar)
for location in location_list:
    df = pd.read_csv("D:\monster_scrapingdata\softdev_" + location + "_20170116.csv", sep='\t', header=0, index_col=0)
    %sql DROP TABLE IF EXISTS df
    %sql PERSIST df
    %sql INSERT INTO ALLPOST SELECT keyword, location, link, post_text FROM df 

In [ ]:
techlist=["javascript","sql", 'java', 'c#', 'php', 'c++', 'c', 'node.js', 'angularjs', 'ruby', 'rails', 'python',
          'delphi', 'typescript', 'bash', 'groovy', 'powershell', 'html', 'css', 'django','postgresql', 'jquery', 'ios', 'git', 
          'mysql', '.net', 'arrays', 'swift', 'objective-c', 'string', 'json', 'sql server', 'linux', 'regex', 'asp.net', 
         'react', 'spark', 'cassandra', 'raspberry pi', 'wordpress', 'mongodb', 'redis', 'vba', 'hadoop', 'clojure',"visual basic",
          'f#', 'sharepoint', 'rust', 'scala', 'cordova', 'perl', 'salesforce', 'matlab', 'dart', 'coffeescript',
         'haskell', 'windows phone', 'aws', 'gae', 'azure', 'lamp', "lua", "drupal", "elixir", "fortran", "erlang", "pl/sql", 
         "elasticsearch", "coldfusion", "kotlin", "smalltalk", "abap", "elm", "r", "cloud", "sas", "d3", "spss",
         "excel", "tableau", "pig", "mapreduce", "hive", "shark", "oozie", "zookeeper", "flume", "mahout", "android"]

In [ ]:
## Create Topskills for each location ##
## Output in csv ##

topskill=[]
for location in location_list:
    citycount=%sql SELECT Count(*) FROM ALLPOST WHERE location = :location
    for tech in techlist:
        acount=%sql SELECT Count(*) FROM ALLPOST WHERE location = :location AND (post_text LIKE "%,$tech,%" OR post_text LIKE "$tech,%" OR post_text LIKE "%,$tech")
        topskill.append((location, tech, acount[0][0], acount[0][0]/citycount[0][0]))   
    print ("done" + location )
df_csv = pd.DataFrame(topskill, columns=[ "location", "skill", "posting", "percentage"])
df_csv.to_csv(r"D:\Data Incubator\capstone_project\topskill.csv", sep='\t', index=False, encoding='utf-8')

In [ ]:
##Top 5 skills to csv ##

dftop5=df_csv.sort_values("posting", ascending= False).groupby("location").head(5).sort_values(["location", "posting"], ascending = False)
dftop5.to_csv(r"D:\Data Incubator\capstone_project\topfiveskill.csv", sep='\t', index=False, encoding='utf-8')

In [3]:
lonlat_list=[("Waterloo_ON", 43.4643, -80.5204), ("Washington_DC", 38.930176, -77.070503),("Vancouver_BC", 49.246292, -123.116226), 
("Toronto_ON", 43.653908, -79.384293), ("Seattle_WA", 47.608013, -122.335167), ("San-Jose_CA", 37.363949, -121.928940), 
 ("San-Francisco_CA", 37.733795, -122.446747), ("San-Diego_CA", 32.715736, -117.161087), ("Saint-Louis_MO", 38.641884, -90.254845),
("Raleigh_NC", 35.787743, -78.644257), ("Portland_OR", 45.523064, -122.676483), ("New-York_NY", 40.785091, -73.968285),
("Montreal_QC", 45.537258, -73.629768), ("Minneapolis_MN", 44.986656, -93.258133), ("Houston_TX", 29.766083, -95.358810),
("Denver_CO", 39.742043, -104.991531), ("Boston_MA", 42.359215, -71.049149), ("Austin_TX", 30.266926, -97.750519),
("Atlanta_GA", 33.753746, -84.386330)]

In [44]:
## plotly map plot ##
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


df_fig= pd.read_csv(r"D:\Data Incubator\capstone_project\topfiveskill.csv", sep='\t', encoding='utf-8')

grplist=list(df_fig.groupby("location",sort=False))
txts = [ "<br>".join(["%s %d %.1f%%" % (skill, posting, (percentage)*100) for skill, posting, percentage in 
                      df[["skill", "posting", "percentage"]].itertuples(index=False)]) for location, df in grplist]
citynames = [x[0] for x in lonlat_list]
final_txts = ["<b>" + city + "</b><br>" + txt for txt, city in zip(txts, citynames)]
data = [ dict(
    lat = [x[1] for x in lonlat_list],
    lon = [x[2] for x in lonlat_list],
    text = final_txts,
    marker = dict(
        color = "rgb(90, 120, 245)",
        reversescale = True,
        opacity = 0.7,
        size = 10,   
        symbol = 'square',
        line = dict(
            width=1,
            color='rgba(102, 102, 102)')),
    type = 'scattergeo', name='Major Tech City'
) ]


layout = dict(
        title = "Top Skills for Major Cities<br>(Hover for details)",   
        geo = dict(scope = 'north america',
                   showland = True, landcolor = "rgb(212, 212, 212)",
                    subunitcolor = "rgb(253, 242, 233)",
                    countrycolor = "rgb(250, 229, 211)",
                    showlakes = True, lakecolor = "rgb(255, 255, 255)",
                    showsubunits = True, showcountries = True,
                   resolution = 50,
                    projection = dict(type = 'Kavrayskiy7', rotation = dict(lon = -100)),
                    lonaxis = dict(showgrid = True, gridwidth = 0.5, range= [ -127.0, -65.0 ], dtick = 5),
                    lataxis = dict(showgrid = True, gridwidth = 0.5, range= [ 25.0, 50.0 ], dtick = 5)),
        showlegend=True, legend=dict(x=0.75, y=0.05, font=dict(size=12)))

fig = dict( data=data, layout=layout )
iplot( fig, validate=False, filename="top5skillmap" )